In [99]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [100]:
# global variables+
target = "Attrition"

In [101]:
def main():
    # create dataframe with project's data set
    df = pd.read_csv('HR.csv')
    # clean the data
    clean_df = clean_data(df)
    # get features matrix X and target variable y
    X, y = feature_selection(clean_df)
    

In [112]:
def extract_categoricals(df):
    # extract a list of all categorical features for feature engineering
    cat_features = df.select_dtypes(include=['object']).columns.tolist()
    # exclude target variable, attrition
    cat_features.pop(cat_features.index(target))
    return cat_features
    
        

In [108]:
def clean_data(df):
    # drop columns that are not needed
    for col_header in df.columns:
        if col_header not in features and col_header != target:
            df.drop(col_header, axis = 1, inplace=True)         
    # drop rows that contain at least one empty cell
    df.dropna(axis='rows', thresh=1)
    # change Yes or No in Attrition column to 0 and 1
    df[target] = df[target].map({'yes': 1, 'no': 0})
    # feature enigneering for categorical data
    df = pd.get_dummies(data=df, columns=extract_categoricals(df))   
    return df


In [109]:
def feature_selection(df): 
    # delete target variable from features list
    # features = del extract_variables(df)[1:2]
    # split dataset in features and target variable
    X = df[features] # Features Matrix
    y = df[target] # Target variable
    return X, y


In [110]:
def construct_tree(X,y):
    decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
    decision_tree = decision_tree.fit(X.data, y.target)
    tree.plot_tree(decision_tree)

In [113]:
# test
df = pd.read_csv('HR.csv')
# clean the data
clean_df = clean_data(df)
print(clean_df)


# get features matrix X and target variable y
# X, y = feature_selection(clean_df)
# print("X: ", X)
# print("y: ", y)



ValueError: 'Attrition' is not in list